# CityBikes Rest API

This section asks you to fetch JSON data from the [CityBikes REST API](https://api.citybik.es/v2/) to answer some questions about bike share programs.

Please be mindful not to send requests to the server more often than necessary. In particular, make sure you separate code that requests data from the server into a separate cell so that you do not have to repeatedly request data from the server. You will lose points if your requests are not separated into their own cell.

# Question 1

Find all cities with bike share programs in the United States (country code "US"), along with their network ID (for example, "bay-wheels"). How many cities in the U.S. have bike share programs (at least that are in this API)?

In [1]:
import pandas as pd
from pandas.io.json import json_normalize
import requests
import json

response = requests.get("http://api.citybik.es/v2/networks?")


In [2]:
data_networks = response.json()

In [3]:
UScities = []
USids = []

In [4]:
for company in data_networks['networks']:
  if (company['location']['country']) == 'US':
    UScities.append(company['location']['city'])
    USids.append(company['id'])

In [5]:
UScities

['Aspen, CO',
 'Ann Arbor, MI',
 'Austin, TX',
 'Chattanooga, TN',
 'Portland, OR',
 'San Ramon, CA',
 'Boise, ID',
 'Boulder, CO',
 'Santa Monica, CA',
 'Fort Lauderdale, FL',
 'Milwaukee, WI',
 'Buffalo, NY',
 'Washington, DC',
 'Charlotte, NC',
 'Cincinnati, OH',
 'New York, NY',
 'Tampa, FL',
 'Columbus, OH',
 'Denver, CO',
 'Chicago, IL',
 'El Paso, TX',
 'Fort Worth, TX',
 'Fargo, ND',
 'Salt Lake City',
 'Phoenix, AZ',
 'Omaha, NE',
 'Houston, TX',
 'Boston, MA',
 'Philadelphia, PA',
 'Indianapolis, IN',
 'Orlando, FL',
 'Kansas City, MO',
 'Dayton, OH',
 'Madison, WI',
 'Los Angeles, CA',
 'Ketchum / Sun Valley, ID',
 'Minneapolis, MN',
 'San Antonio, TX',
 'University of South Florida, FL',
 'Long Beach, NY',
 'Spartanburg, SC',
 'Topeka, KS',
 'University of Virginia, VA',
 'Atlanta, GA',
 'Miami Beach, FL',
 'San Diego, CA',
 'Des Moines, IA',
 'Greenville, SC',
 'Pittsburgh',
 'San Francisco Bay Area, CA',
 'Las Vegas, NV',
 'Oklahoma, OK']

In [6]:
USids

['we-cycle',
 'arborbike',
 'austin',
 'bike-chattanooga',
 'biketown',
 'britebikes',
 'boise-greenbike',
 'boulder',
 'breeze-bike-share',
 'broward',
 'bublr-bikes',
 'reddy-bike-share',
 'capital-bikeshare',
 'charlotte',
 'cincy-red-bike',
 'citi-bike-nyc',
 'coast-bike-share',
 'cogo',
 'denver',
 'divvy',
 'elpaso',
 'fortworth',
 'greatrides',
 'greenbikeslc',
 'grid-bike-share',
 'omaha',
 'houston',
 'blue-bikes',
 'indego',
 'indiana-pacers-bikeshare',
 'juice-bike-share',
 'kansascity',
 'linkdayton',
 'madison',
 'metro-bike-share',
 'mountain-rides-bike-share',
 'nice-ride',
 'sanantonio',
 'share-a-bull-bikes',
 'sobi-long-beach',
 'spartanburg',
 'topeka-metro-bikes',
 'ubike',
 'relay-atlanta',
 'decobike-miami-beach',
 'decobike-san-diego',
 'desmoines',
 'greenville',
 'healthy-ride-pittsburgh-pittsburgh',
 'bay-wheels',
 'rtc-bike-share',
 'spokies']

In [7]:
print(len(UScities))

52


There are 52 US cities with bike share programs. 

In [8]:
import pandas as pd
from pandas.io.json import json_normalize
import requests
import json


response = requests.get("http://api.citybik.es/v2/networks")

# Extract the networks with country code "US"
us_networks = [network for network in response.json()["networks"] if network["location"]["country"] == "US"]
us_network_pairs = []

# Print the network ID and city name for each US network
for network in us_networks:
    network_id = network["id"]
    network_location = network["location"]["city"]
    us_network_pairs.append((network_id, network_location))
    print(f"ID: {network_id:40} Location: {network_location:40}")

# Print the total number of cities with bike share programs in the US
print(f"\nTotal cities in the US with bike share programs: {len(us_network_pairs)}")  

ID: we-cycle                                 Location: Aspen, CO                               
ID: arborbike                                Location: Ann Arbor, MI                           
ID: austin                                   Location: Austin, TX                              
ID: bike-chattanooga                         Location: Chattanooga, TN                         
ID: biketown                                 Location: Portland, OR                            
ID: britebikes                               Location: San Ramon, CA                           
ID: boise-greenbike                          Location: Boise, ID                               
ID: boulder                                  Location: Boulder, CO                             
ID: breeze-bike-share                        Location: Santa Monica, CA                        
ID: broward                                  Location: Fort Lauderdale, FL                     
ID: bublr-bikes                         

# Question 2

Construct a `DataFrame` containing data about all bike stations in all networks in the United States. Save this `DataFrame` to disk using `.to_csv()`. (You will need it in Part B of this lab.)

Use this `DataFrame` to determine the total number of bicycles in bike share programs across the United States? You may assume that the number of bikes at a station is the number of empty spaces, plus the number of available bikes.

In [9]:
stations = []

for pair in us_network_pairs:
    url = "http://api.citybik.es/v2/networks/" + pair[0]
    data = requests.get(url).json()

    for station in data["network"]["stations"]:           
        stations.append({"city": pair[1],
                        "network": pair[0],
                        "name": station["name"],
                        "free_bikes": station["free_bikes"],
                        "empty_slots": station["empty_slots"],
                        "timestamp": station["timestamp"],
                        "longitude": station["longitude"],
                        "latitude": station["latitude"],
                        "id": station["id"]})

stations_df = pd.DataFrame(stations)
stations_df["total_bikes"] = stations_df["free_bikes"] + stations_df["empty_slots"]
total_bikes = stations_df["total_bikes"].sum()
stations_df.to_csv("bike_stations.csv", index = False)
print(f"total bikes - {total_bikes}")

total bikes - 145814


# Question 3

You have just finished touring Coit Tower in San Francisco, which is located at latitude 37.802747 and longitude -122.405861. Using your `DataFrame` from Question 2, find the nearest bike station with an available bike, based on taxicab distance (a.k.a. Manhattan distance).

_Hint:_ You can check your answer using Google Maps!

In [10]:
available = stations_df[stations_df["free_bikes"] > 0]
distances = [abs(37.802747 - available["latitude"]) + abs((-122.405861 - available["longitude"]))]
print(distances[0].sort_values())
stations_df.loc[7651]

7635     0.004650
7885     0.004823
8015     0.004837
7871     0.005675
7851     0.007277
          ...    
5901    56.226871
5888    56.230268
5813    56.230455
5815    56.231958
5866    56.235152
Length: 5733, dtype: float64


city                 San Francisco Bay Area, CA
network                              bay-wheels
name                       Horton St at 40th St
free_bikes                                   12
empty_slots                                  11
timestamp           2023-03-09T05:46:21.950000Z
longitude                            -122.28761
latitude                              37.829705
id             5a3ef6c03f1ce19c588c078270170a18
total_bikes                                  23
Name: 7651, dtype: object

## Submission Instructions

- Copy this notebook to your own Drive, if you have not already.
- Restart this notebook and run the cells from beginning to end. 
  - Go to Runtime > Restart and Run All.
- Rename this notebook by clicking on "DATA 301 Lab 6A - YOUR NAMES HERE" at the very top of this page. Replace "YOUR NAMES HERE" with the first and last names of you (and your partners, for Phase 2).
- Get the link to your notebook:
  - Click on "Share" at the top-right. 
  - Change the settings to "Anyone with the link can view". 
  - Copy the sharing link into Canvas.